In [30]:
import pyspark
from pyspark.sql import SparkSession

In [41]:
URL='https://drive.google.com/uc?export=download&id=1vcb_HBWsOSKW4XxhLfRpGlLzBLwHlGWJ'
!wget --no-check-certificate "$URL" -O multi_source_demo.zip

--2024-04-02 18:10:46--  https://drive.google.com/uc?export=download&id=1vcb_HBWsOSKW4XxhLfRpGlLzBLwHlGWJ
Resolving drive.google.com (drive.google.com)... 64.233.166.102, 64.233.166.113, 64.233.166.138, ...
Connecting to drive.google.com (drive.google.com)|64.233.166.102|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1vcb_HBWsOSKW4XxhLfRpGlLzBLwHlGWJ&export=download [following]
--2024-04-02 18:10:46--  https://drive.usercontent.google.com/download?id=1vcb_HBWsOSKW4XxhLfRpGlLzBLwHlGWJ&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 74.125.133.132, 2a00:1450:400c:c07::84
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|74.125.133.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2433 (2.4K) [text/html]
Saving to: ‘multi_source_demo.zip’

multi_source_demo.z 100%[===================>]   2.38K  --.-KB/s    in 0.00

In [43]:
!unzip multi_source_demo.zip

Archive:  multi_source_demo.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of multi_source_demo.zip or
        multi_source_demo.zip.zip, and cannot find multi_source_demo.zip.ZIP, period.
